In [4]:
from pynq import Overlay
from matplotlib import pyplot as plt
from pynq import allocate
import pynq.lib.dma
import numpy as np
from numpy import fft
import random as rand
import operator

In [15]:
overlay = Overlay('fft_bitstream/fft.bit')
dma = overlay.FFT.axi_dma_0

In [16]:
def makeSongDict(listOfSongs):
    fs_rate, signal_original = wavfile.read(filename)
    total_time = int(np.floor(len(signal_original)/fs_rate))
    sample_range = np.arange(0,total_time,time_period)
    total_samples = len(sample_range)
    
    for i in sample_range:
        sample_start = int(i*fs_rate)
        sample_end = int((i+time_period)*fs_rate)
        signal = signal_original[sample_start:sample_end]
    
    in_buffer = signal
    run_kernel()
    
    FFT_side = out_buffer
    output_array = []
    # Reduce to 0-5000 Hz
    bucket_size = 20
    buckets = 16

    FFT_side = FFT_side[0:bucket_size*buckets]

    # Combine frequencies into buckets
    FFT_side = np.array([int(sum(FFT_side[current: current+bucket_size])) for current in range(0, len(FFT_side), bucket_size)])

    # FFT_side: Normalize (0-1)
    max_value = max(FFT_side)
    if (max_value != 0):
        FFT_side_norm = FFT_side / max_value

    # Append to output array
    output_array.append(FFT_side_norm)
    print(output_array)
    

In [18]:
makeSongDict('test.wav')

NameError: name 'wavfile' is not defined

In [5]:
in_buffer = allocate(shape=(1024,),dtype=np.uint32)
out_buffer = allocate(shape=(1024,), dtype = np.uint32)
in_buffer[:] = [rand.randrange(0,10) for i in range(0, 1024)]


In [6]:
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [ ]:
%%timeit
run_kernel()
print(out_buffer)

In [11]:
FFT_side = out_buffer
output_array = []
# Reduce to 0-5000 Hz
bucket_size = 20
buckets = 16

FFT_side = FFT_side[0:bucket_size*buckets]

# Combine frequencies into buckets
FFT_side = np.array([int(sum(FFT_side[current: current+bucket_size])) for current in range(0, len(FFT_side), bucket_size)])

# FFT_side: Normalize (0-1)
max_value = max(FFT_side)
if (max_value != 0):
    FFT_side_norm = FFT_side / max_value

# Append to output array
output_array.append(FFT_side_norm)

In [13]:
print(output_array)

[array([1.        , 0.62534051, 0.56300863, 0.68796829, 0.75052353,
       0.68823175, 0.62523551, 0.43822269, 0.56323772, 0.6879177 ,
       0.81282677, 0.3759481 , 0.62558583, 0.56320717, 0.75044908,
       0.62550755])]


In [5]:
overlayCompare = Overlay('compare_bitstream/compare.bit')
overlayCompare?
out_buffer_compare = allocate(shape=(1024,), dtype = np.uint32)

In [9]:
def run_compare():
    overlayCompare.processing_system7_0(output_array, output_array)
    recvchannel.transfer(out_buffer_compare)
    sendchannel.wait()
    recvchannel.wait()

In [10]:
run_compare()
print(out_buffer_compare)

KeyError: 'phys_addr'